# Fine tunning BART


In [1]:
from transformers import (
    DataCollatorForSeq2Seq,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    AutoTokenizer,
)
import evaluate
from datasets import Dataset
import numpy as np
import pandas as pd
import torch
from torch.utils.data import random_split

from tqdm import tqdm

import warnings

In [2]:
MANUAL_SEED = 42

warnings.filterwarnings("ignore")

## Data loading and preprocessing


In [3]:
df = pd.read_csv("../data/raw/dataset_xs.csv")
print(f"{len(df)=}")
df.head()

len(df)=9462


,toxic,nontoxic
0,I like that shit.,I love it.
1,"Now, I understand you got your grievances with...","I understand you don't have to cut your bills,..."
2,Damn It!,"oh, my God."
3,"Help me, you cunt!","Aitchi, help me!"
4,Look at that shit.,look at this.


In [13]:
train_seq, val_seq, test_seq = random_split(
    range(len(df)),  # type: ignore
    [0.85, 0.1, 0.05],
    generator=torch.Generator().manual_seed(MANUAL_SEED),
)
train_indices, val_indices, test_indices = (
    list(train_seq.indices),
    list(val_seq.indices),
    list(test_seq.indices),
)
print(f"{len(train_indices)=}")
print(f"{len(val_indices)=}")
print(f"{len(test_indices)=}")

len(train_indices)=8043
len(val_indices)=946
len(test_indices)=473


## Model definition


In [11]:
checkpoint = "eugenesiow/bart-paraphrase"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [9]:
BATCH_SIZE = 16

PREFIX = "paraphrase following to be nontoxic: \n"
MAX_LENGTH = 128

In [7]:
def preprocess_function(data):
    inputs = [PREFIX + data_point for data_point in data["toxic"]]
    targets = data["nontoxic"]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=MAX_LENGTH, truncation=True
    )
    return model_inputs


def post_process_text(predictions, targets):
    predictions = [pred.strip() for pred in predictions]
    targets = [label.strip() for label in targets]
    return predictions, targets


BLEU_METRIC = evaluate.load("bleu")


def compute_metrics(batch):
    predictions, targets = batch
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    targets = np.where(targets != -100, targets, tokenizer.pad_token_id)  # type: ignore
    decoded_targets = tokenizer.batch_decode(targets, skip_special_tokens=True)

    decoded_predictions, decoded_targets = post_process_text(
        decoded_predictions, decoded_targets
    )

    result = {}
    metrics = BLEU_METRIC.compute(
        predictions=decoded_predictions, references=decoded_targets
    )
    if metrics is not None:
        result.update({"bleu": metrics["bleu"]})

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [14]:
train_dataset = Dataset.from_pandas(df.iloc[train_indices]).map(
    preprocess_function, batched=True
)
val_dataset = Dataset.from_pandas(df.iloc[val_indices]).map(
    preprocess_function, batched=True
)

Map:   0%|          | 0/8043 [00:00<?, ? examples/s]

Map:   0%|          | 0/946 [00:00<?, ? examples/s]

## Train model


In [12]:
training_args = Seq2SeqTrainingArguments(
    output_dir="../models/train_data/bart",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    report_to="none",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # type: ignore
    eval_dataset=val_dataset,  # type: ignore
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.459700,1.398838,0.399000,12.200800
2,0.951500,1.331711,0.394200,9.972500
3,0.679400,1.351861,0.439300,10.750500
4,0.470300,1.496082,0.440300,11.693400
5,0.335700,1.632915,0.449600,11.030700
6,0.245900,1.775123,0.453900,9.987300
7,0.184400,1.883072,0.459200,10.635300
8,0.139200,1.965904,0.450300,10.964100
9,0.104800,2.019606,0.457700,10.515900
10,0.079100,2.160270,0.456600,11.470400


TrainOutput(global_step=5030, training_loss=0.4625845614768876, metrics={'train_runtime': 3270.0551, 'train_samples_per_second': 24.596, 'train_steps_per_second': 1.538, 'total_flos': 5941157476638720.0, 'train_loss': 0.4625845614768876, 'epoch': 10.0})

In [ ]:
trainer.save_model("../models/bart")

## Test model


In [15]:
model = AutoModelForSeq2SeqLM.from_pretrained("../models/bart")
model.eval()
model.config.use_cache = False

In [16]:
test_df = df.iloc[test_indices]
test_df.head()

,toxic,nontoxic
7898,You know how the Romans settled this shit?,do you know how the Romans handled this?
5283,What kind of shit do you talk?,what are you talking about?
6611,Idiot. There's a storage shed near the back.,there's a warehouse in the back.
3187,Come get her! Goddamn you!,come and get her!
8466,"You fucker Hello, post office?","hello, post office?"


In [17]:
def detoxify(model, prompt):
    inference_request = PREFIX + prompt
    input_ids = tokenizer(inference_request, return_tensors="pt").input_ids
    outputs = model.generate(input_ids=input_ids)
    return tokenizer.decode(outputs[0], skip_special_tokens=True, temperature=0)


detoxify(model, "shut up, man")

'quiet, man.'

In [ ]:
model_answers = []
for i, r in tqdm(test_df.iterrows(), total=len(test_df)):
    model_answers.append(detoxify(model, r["toxic"]))


test_df["generated"] = model_answers

100%|██████████| 473/473 [18:27<00:00,  2.34s/it]


In [ ]:
test_df.head()

,toxic,nontoxic,generated
7898,You know how the Romans settled this shit?,do you know how the Romans handled this?,you know how the Romans decided this?
5283,What kind of shit do you talk?,what are you talking about?,what are you talking about?
6611,Idiot. There's a storage shed near the back.,there's a warehouse in the back.,there's a storage shed in the back.
3187,Come get her! Goddamn you!,come and get her!,come get her!
8466,"You fucker Hello, post office?","hello, post office?","hello, mailman?"


## Save results


In [17]:
test_df.to_csv("../data/generated/bart.csv", index=False)